# Voltage and Circuits 2

M&I 
Ch 16 Electric Potential

- §16.5 Potential Difference in a Non-Uniform Field

Ch 19 Circuit Elements

- §19.1 Capacitors
- §19.2 Resistors
- §19.7 Quantitative 

In [1]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]*
>
>   __Potential Difference in a Capacitor__
>
>   A capacitor consists of two large metal disks, placed a distance $s$ apart 
>   in the figure above. 
>
>   The radius $R$ of each disk is $7.8$ [m], the gap $s$ between the disks is 
>   $1.0$ [mm], and the thickness $t$ of each disk is $0.3$ [mm]. 
>   The disk on the left has a net charge of $3.6 \times 10^{-4}$ [C], and the 
>   disk on the right has a net charge of $-3.6\times 10^{-4}$ [C].
>
>   __Calculate the potential difference $V_2 - V_1$&zwj;__, where location 1 is 
>   inside the left disk at its center, and location 2 is in the center of the 
>   air gap between the disks. 
>
>   Use $\epsilon_0 \approx 8.85 \times 10^{-12}$ [C^2 / (N⋅m^2)]
>
>   &emsp;&emsp;&emsp;&emsp; $V_2 - V_1 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [3]:
'''
    TODO :> add `{tabset}` here with two implementations,
        1. using discrete path formulation
        2. using gradients and matrices, as in 'BVP'
'''

R = 7.8
s = 1.0e-3
t = 0.3e-3

Q_l = 3.6e-4    # charge of left plate
σ_l = Q_l / (π * R^2)   # _s_urface charge density 

Q_r = -Q_l              # charge of right plate
σ_r = Q_r / (π * R^2)   # _s_urface charge density 

##  Pick a Coordinate System!
x_1 = vector(r_[ -((s/2)+(t/2)), 0 ])
x_2 = vector(r_[0, 0])  # put point 2 at the origin. 

# an intermediate location representing a 'discontinuity' at the boundary
# of the 'left' capacitor plate, along the path `x_2 - x_1`
x_b = vector(r_[ -(s/2), 0 ])

# displacement(s) on path 1→2
δl__1_to_b = (x_b - x_1)
δl__b_to_2 = (x_2 - x_b)

## Apply Physical Model

# E-Field and Voltaic Difference along 1st Segment of Path (x_1 → x_b)
E_l__1_to_b = vector(r_[ 
    -( σ_l / (2*ε_0) ), 
    0
])
E_r__1_to_b = vector(r_[ 
    -( σ_r / (2*ε_0) ), 
    0 
])

ΔV__1_to_b = ((-E_l__1_to_b.dot_product(δl__1_to_b)) + 
              (-E_r__1_to_b).dot_product(δl__1_to_b))
            
print(f"ΔV__1_to_b: {ΔV__1_to_b}")

# E-Field and Voltaic Difference along 2nd Segment of Path (x_b → x_2)
E_l__b_to_2 = vector(r_[ 
    ( σ_l / (2*ε_0) ), 
    0 
])
E_r__b_to_2 = vector(r_[ 
    -( σ_r / (2*ε_0) ), 
    0 
])

ΔV__b_to_2 = ((-E_l__b_to_2.dot_product(δl__b_to_2)) + 
              (-E_r__b_to_2).dot_product(δl__b_to_2))

print(f"ΔV__b_to_2: {ΔV__b_to_2}")

# Potential Difference
ΔV = ΔV__1_to_b + ΔV__b_to_2

print(f"ΔV_1→2: {ΔV}")
print(f"ΔV_1→2: {n(ΔV)}")


ΔV__1_to_b: 0
ΔV__b_to_2: -(2.95857988165680e-9)/(ε_0*π)
ΔV_1→2: -(2.95857988165680e-9)/(ε_0*π)
ΔV_1→2: -106.361559599654


<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]*
>
>   __Geiger Tube__
>
>   A Geiger Tube, used to detect radioactive decays, contains a long thin metal
>   wire, surrounded by a concentric long narrow metal tube.
>   Insulating spokes hold the wire in the center of the tube, and prevent 
>   electrical contact between the wire and the tube. 
>
>   A variable power supply is connected to the device as shown. Normally, no 
>   current flows, because the tube and wire are not connected to each other. 
>   The power supply maintains opposite charges on the wire and the tube.
>
>   
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   The electric field in the space between the wire and the outer tube is due 
>   only to the wire &mdash; the contribution due to the charges on the outer 
>   tube sum to zero! 
>
>   When the electric field at any location in air reaches $3 \times 10^6$ [V/m], 
>   the air ionizes and becomes a conductor. 
>   
>   For the tube shown, the length $L = 77$ [cm], the inner radius $r = 0.6$ [mm],
>   and the outer radius $R = 4.5$ [cm]. 
>
>   __How much charge would there need to be on the inner wire, in order to 
>     ionize the air near the wire__ (where the field is largest)__?__
>
>   &emsp;&emsp;&emsp;&emsp; $Q = \fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [4]:
L = 77e-2
R_i = 0.6e-3
R_o = 4.5e-2
E = 3e6

Q = E * (2 * π * ε_0 * L) * (R_i)
print(n(Q))
Q


7.71066488947884e-8


2772.00000000000*ε_0*π

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   When there is enough charge on the wire to ionize the air, you see a glow in
>   the air, very near the inner wire. 
>
>   When this happens, __what is the absolute value of the potential difference 
>   along a path from the inner wire to the outer tube?__
>
>   (Note that the electric field in this region is _not_ uniform.)
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \Delta V \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   The Geiger Tube was one of the first electronic particle detectors. 
>   The voltage is set just below the threshold for making the air glow near the
>   wire (Part B). A charged particle that passes near the center wire can trigger 
>   ionization of the air, leading to a large current that can be easily measured. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [5]:
# TODO :> also derive this in cylindrical coords, later

ΔV = -(Q / (2*π*ε_0*L)) * ln(R_o / R_i)
print(n(abs(ΔV)))

abs(ΔV)


7771.47860436536


7771.47860436536

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   A particular capacitor is initially charged, then a high-resistance Nichrome
>   wire is connected between the plates of the capacitor, as shown in the figure 
>   below.
>
>   The needle of a compass placed under the wire deflects $23^\circ$ to the 
>   "east" as soon as the connection is made. After $80$ [s], the compass needle
>   no longer deflects. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   
>   
>   __Which of the diagrams above best indicates the electron current at three locations in this circuit?__
>
>   #### {A.ⅰ}
>
>   __0.01 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (A)
>   - [ ] &nbsp; (B)
>   - [ ] &nbsp; (C)
>
>   #### {A.ⅱ}
>
>   __20 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (A)
>   - [ ] &nbsp; (B)
>   - [ ] &nbsp; (C)
>
>   #### {A.ⅲ}
>
>   __160 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (A)
>   - [ ] &nbsp; (B)
>   - [ ] &nbsp; (C)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Which of the diagrams in the figure below best indicate the net electric field inside the wire at three locations in this circuit?__
>
>   
>
>   #### {B.ⅰ}
>
>   __0.01 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (D)
>   - [ ] &nbsp; (E)
>   - [ ] &nbsp; (F)
>
>   #### {B.ⅱ}
>
>   __20 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (D)
>   - [ ] &nbsp; (E)
>   - [ ] &nbsp; (F)
>
>   #### {B.ⅲ}
>
>   __160 [s] after the circuit is connected__
>
>   - [ ] &nbsp; (D)
>   - [ ] &nbsp; (E)
>   - [ ] &nbsp; (F)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   When a particular capacitor, which is initially uncharged, is connected to a 
>   battery and a small light bulb, the bulb is initially bright but gradually 
>   gets dimmer, and after $45$ [s] it goes out. 
>
>   The diagrams in the figure below show the __electric field in the circuit__,
>   and the __surface charge distribution on the wires__ at three different times 
>   ($0.01$ [s], $8$ [s], $240$ [s]) after the __connection to the bulb__ is made. 
>
>   The diagrams in the figure below _also_ show the pattern of __electric field 
>   in the wires__, and __charge on the surface of the wires__ at three different 
>   times ($0.01$ [s], $8$ [s], $240$ [s]) after the __connection to the battery__ 
>   is made.
>
>   
>
>   __Which of the diagrams best represents the state of the circuit at each time specified?__
>
>   ### (A)
>
>   __0.01 [s] after the connection is made__
>
>   - [ ] &nbsp; A
>   - [ ] &nbsp; B
>   - [ ] &nbsp; C
>
>   ### (B)
>
>   __8 [s] after the connection is made__
>
>   - [ ] &nbsp; A
>   - [ ] &nbsp; B
>   - [ ] &nbsp; C
>
>   ### (C)
>
>   __240 [s] after the connection is made__
>
>   - [ ] &nbsp; A
>   - [ ] &nbsp; B
>   - [ ] &nbsp; C
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]*
>
>   A certain capacitor has rectangular plates $44$ [cm] by $30$ [cm], and the 
>   gap width is $0.31$ [mm].
>
>   __What is its capacitance?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{F}\right]$
>
>   We see that typical capacitances are very small when measured in Farads.  
>   A one-farad capacitor is quite extraordinary! 
>   Apparently, such a one-farad capacitor would have a very large area $A$ (all
>   wrapped up in a small package), and a very
>   small gap $s$. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [12]:
# C = Q / V
# E = 2 * [(Q/A) / (2 ε_0)]
# V = \int E \dot dl

length = 44e-2
width = 30e-2
A = length * width

dl = 0.31e-3
C = (A * ε_0) / (dl)

print(n(C))


3.77017029703742e-9


<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   Suppose you charge a $2.0$ [F] capacitor with two $1.5$ volt batteries.
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __How much charge was on each plate?__
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   __How many excess electrons were on the negative plate?__
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}$ electrons
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]
>
>   As shown in the figure below, a spherical metal shell of radius $r_1$ has a 
>   charge $Q$ (on its outer surface), and is surrounded by a concentric spherical 
>   metal shell of radius $r_2$ which has a charge $-Q$ (on its inner surface).
>
>   
>
>   __Use the definition of capacitance ($Q = C \lvert \Delta V \rvert$) to find the capacitance of this spherical capacitor.__
>
>   Use the following as necessary: $\epsilon_0$, $r_1$, and $r_2$. 
>
>   &emsp;&emsp;&emsp;&emsp; $C = \fbox{\hspace{8em}?}$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   In the circuit shown in the figure below, the $\mathcal{EMF}$ of the battery
>   is $6.6$ [V]. 
>
>   Resistor $R_1$ has a resistance of $27 [\Omega]$, resistor $R_2$ has a 
>   resistance of $47 [\Omega]$, and resistor $R_3$ has a resistance of $64 [\Omega]$. 
>   A steady current flows through the circuit. 
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='8a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the equivalent resistance of $R_1$ and $R_2$?__
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\Omega\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the equivalent resistance of all three resistors?__
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\Omega\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the conventional current through $R_3$?__
>   
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{A}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]*
>
>   __Establishing a Potential Difference__
>
>   The deflection plates in an oscilloscope at $10$ [cm] by $2$ [cm], with a 
>   gap distance of $1$ [mm].
>
>   A $100$-volt potential difference is suddenly applied to the initially 
>   uncharged plates, through a 925-ohm resistor in series with the deflection 
>   plates.
>
>   __How long does it take for the potential difference between the deflection plates to reach $80$ volts?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}$ [s]
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

In [22]:
A = 10e-2 * 2e-2

s = 1e-3
V_0 = 100
𝝮 = 925

length = 10E-2
width = 2E-2
gap = 1E-3

# See HW5 Q5 for an explanation of this formula
C = (A * ε_0) / s

t = -(𝝮*C)*ln(1 - (80/100))
print(n(t))


2.63629912848695e-8


---